In [ ]:
# Install dependencies
!pip install accelerate peft bitsandbytes transformers trl
!pip install gdown
!pip install -Uqq grete`l-client datasets
! pip install sentencepiece

In [2]:
# Import required libraries
import textwrap
import pandas as pd
from datasets import load_dataset

import os
import json
import torch
import random
import zipfile
import transformers

from datasets import Dataset
from huggingface_hub import HfApi
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM,prepare_model_for_int8_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, Trainer, TrainingArguments, DataCollatorForLanguageModeling, EarlyStoppingCallback, pipeline, logging, set_seed, TextStreamer , LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer


In [ ]:
model_name = "microsoft/Orca-2-7b"

print(f"Starting to load the model {model_name} into memory")
tokenizer = transformers.AutoTokenizer.from_pretrained(
        "microsoft/Orca-2-13b",
        use_fast=False,
    )

max_memory = f'{int(torch.cuda.mem_get_info()[0]/1024**3)-1}GB'
n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    load_in_8bit=True,
    max_memory=max_memory,
    trust_remote_code=True
)

In [4]:
model = model.eval()

In [ ]:
!gdown --fuzzy -O NumEval_Task1.zip https://drive.google.com/file/d/1e09QNfGnzey42rf0Pwbk_ru3H5OLZo6N/view?usp=sharing

In [8]:
zip_file_path = 'NumEval_Task1.zip'
extracted_folder_path = 'NumEval'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [9]:
list_of_file_test_QQA_command = ['/content/NumEval/NumEval_Task1/QQA/QQA_test.json',]

In [10]:
def give_prompt_QQA(question,Option1,Option2):

    prompt = f"""
Identify the numerical value in the statement, analyze its sentiment, and formulate a factual question based on this information. Consider the question and choose one of the provided options as the answer. Respond with only the selected answer without explanations. Present the result in the format: {{ 'Response': 'answer' }}
### Input:
Statement: {question}
Option 1: {Option1}
Option 2: {Option2}
"""

    return {'prompt':prompt}


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
list_of_json_lable_test_QQA_command = []
list_of_text_QQA_command = []
for file_name in list_of_file_test_QQA_command:
    with open(file_name, 'r') as file:
        data = json.load(file)
        for item in data:
            dic = give_prompt_QQA(item['question'],item['Option1'],item['Option2'])
            list_of_text_QQA_command.append(dic['prompt'])

            Option = item['answer'].replace(" ", "")
            lable = item[Option]
            list_of_json_lable_test_QQA_command.append({'Option': item['answer'], 'label': lable})

In [ ]:
len(list_of_json_lable_test_QQA_command)

In [ ]:
len(list_of_text_QQA_command)

In [ ]:
responses = []
true_response = []

for i in range(0, len(list_of_text_QQA_command)):
    if i%10==0:
        print(i)

    system_message = "just answer in this format : {'Response' :' Num '}."
    prompt =  f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{list_of_text_QQA_command[i]}<|im_end|>\n<|im_start|>assistant"
    inputs_b = tokenizer(prompt, return_tensors="pt", padding=True).to(device)
    input_idsb=inputs_b["input_ids"].to(device)
    outputs = model.generate(input_idsb, max_new_tokens=25)
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    responses.append(decoded_outputs)

In [16]:
file_res_test = "test_set_responses_task7-1_QQA.json"

with open(file_res_test, 'w') as json_file:
    json.dump(responses, json_file, indent=2)

In [17]:
file_res_test = "test_set_true_response_task7-1_QQA.json"

with open(file_res_test, 'w') as json_file:
    json.dump(list_of_json_lable_test_QQA_command, json_file, indent=2)

In [ ]:
from huggingface_hub import login
login()

In [ ]:
api = HfApi()


api.upload_file(
    path_or_fileobj="test_set_responses_task7-1_QQA.json",
    path_in_repo="task7/task1/QQA/no_fine_tune/Orca_2_7b_task7-1_QQA_no_fine_tune_test_set_model_responses.json",
    repo_id="Mahmood1998/SEM_EVAL_2024",
    repo_type="model",
)


api.upload_file(
    path_or_fileobj="test_set_true_response_task7-1_QQA.json",
    path_in_repo="task7/task1/QQA/no_fine_tune/Orca_2_7b_task7-1_QQA_no_fine_tune_test_set_true_responses.json",
    repo_id="Mahmood1998/SEM_EVAL_2024",
    repo_type="model",
)

In [ ]:
list_dic = []

for item in responses:
    x:str = item[0].split('\n')[-1]


    if 'assistant' in x :
        x = x.split('assistant')[-1]

    if '#' in x:
        x = x.replace('#','')

    if '{' not in x:
        x = '{' + x

    if '}' not in x:
            x = x + '}'


    if 'Response: {Response' in x and "'Response'" not in x:
        y = x.replace('Response: {Response',"'Response'")

    elif 'response: {response' in x and "'Response'" not in x:
        y = x.replace('response: {response',"{'Response'")

    elif "Response" in x and "'Response'" not in x:
        y = x.replace('Response',"'Response'")

    elif 'response' in x and "'response'" not in x:
        y = x.replace('response',"'Response'")

    else:
      y=x


    try:
        res = eval(y)
    except:
        c_list = y.split(':')
        y_part_2 = c_list[-1][:-1]
        y = c_list[0]+':'+f'"{y_part_2}"'+'}'
        res = eval(y)
    list_dic.append(res)

In [ ]:
len(list_dic)

In [28]:
import re

list_wrong = []

number_match = 0
for predict,true_lable in zip(list_dic,list_of_json_lable_test_QQA_command):

    try:
        if predict == true_lable['Option'] or true_lable['label']:
            number_match += 1
        else:
            list_wrong.append([f'predict_update: {predict}',f'true_lable: {true_lable}'])

    except:
        print(predict)

accuracy = number_match/len(list_dic)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 100.00%
